# Tensorflow Benchmarking (Linux Native)

`Keras-Tensorflow` benchmarking for Ryzen 7950HS `CPU` under Fedora 39 WS with Mobile NVIDIA RTX 4060 `GPU`.

In [1]:
import tensorflow

2024-01-05 18:05:33.177229: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-05 18:05:33.451487: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 18:05:33.451712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 18:05:33.463204: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 18:05:33.492370: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tensorflow.config.experimental.list_physical_devices()

2024-01-05 18:05:38.759831: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-05 18:05:38.857621: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-05 18:05:38.857840: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tensorflow.test.is_built_with_cuda()

True

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [6]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 8s 0us/step


In [10]:
# scaling image values between 0-1
X_train_scaled = X_train/255
X_test_scaled = X_test/255
# one hot encoding labels
y_train_encoded = keras.utils.to_categorical(y_train, num_classes = 10, dtype = 'float32')
y_test_encoded = keras.utils.to_categorical(y_test, num_classes = 10, dtype = 'float32')

In [11]:
def get_model():
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(32,32,3)),
        keras.layers.Dense(3000, activation='relu'),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dense(10, activation='sigmoid')    
    ])
    model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [12]:
get_model().summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 3072)              0         
                                                                 
 dense_6 (Dense)             (None, 3000)              9219000   
                                                                 
 dense_7 (Dense)             (None, 1000)              3001000   
                                                                 
 dense_8 (Dense)             (None, 10)                10010     
                                                                 
Total params: 12230010 (46.65 MB)
Trainable params: 12230010 (46.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
#Comment out for CPU benchmarking
%timeit -n1 -r1
# CPU
with tf.device('/CPU:0'):
    model_gpu = get_model()
    model_gpu.fit(X_train_scaled, y_train_encoded, epochs = 10)

Epoch 1/10
   9/1563 [..............................] - ETA: 24s - loss: 2.3731 - accuracy: 0.1146 

2024-01-05 18:08:09.577910: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f0bf400b380 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-01-05 18:08:09.578075: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1704474489.599713    6275 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-01-05 18:08:09.599952: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-01-05 18:08:09.600035: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-01-05 18:08:09.617165: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.


  81/1563 [>.............................] - ETA: 23s - loss: 2.1649 - accuracy: 0.2110

2024-01-05 18:08:10.665494: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-01-05 18:08:10.715186: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.


1563/1563 [==============================] - 27s 17ms/step - loss: 1.8182 - accuracy: 0.3540
Epoch 2/10
1563/1563 [==============================] - 29s 19ms/step - loss: 1.6273 - accuracy: 0.4266
Epoch 3/10
1563/1563 [==============================] - 29s 18ms/step - loss: 1.5448 - accuracy: 0.4551
Epoch 4/10
1563/1563 [==============================] - 26s 17ms/step - loss: 1.4851 - accuracy: 0.4761
Epoch 5/10
1563/1563 [==============================] - 28s 18ms/step - loss: 1.4331 - accuracy: 0.4968
Epoch 6/10
1563/1563 [==============================] - 29s 18ms/step - loss: 1.3924 - accuracy: 0.5078
Epoch 7/10
1563/1563 [==============================] - 28s 18ms/step - loss: 1.3529 - accuracy: 0.5231
Epoch 8/10
1563/1563 [==============================] - 28s 18ms/step - loss: 1.3177 - accuracy: 0.5369
Epoch 9/10
1563/1563 [==============================] - 29s 18ms/step - loss: 1.2846 - accuracy: 0.5496
Epoch 10/10
1563/1563 [==============================] - 30s 19ms/step - lo

In [14]:
%timeit -n1 -r1
# GPU
with tf.device('/GPU:0'):
    model_gpu = get_model()
    model_gpu.fit(X_train_scaled, y_train_encoded, epochs = 10)

Epoch 1/10


2024-01-05 18:13:48.750355: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-05 18:13:48.834800: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f0bf00d1f90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-05 18:13:48.834884: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-01-05 18:13:48.845068: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


1563/1563 [==============================] - 6s 3ms/step - loss: 1.8131 - accuracy: 0.3520
Epoch 2/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.6235 - accuracy: 0.4266
Epoch 3/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.5433 - accuracy: 0.4571
Epoch 4/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.4814 - accuracy: 0.4768
Epoch 5/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.4317 - accuracy: 0.4986
Epoch 6/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.3877 - accuracy: 0.5122
Epoch 7/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.3492 - accuracy: 0.5255
Epoch 8/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.3166 - accuracy: 0.5360
Epoch 9/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.2827 - accuracy: 0.5498
Epoch 10/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.2512 - accurac